#### Competition : https://www.kaggle.com/competitions/playground-series-s5e6
##### notebook EDA  : https://www.kaggle.com/code/mohamedabder/1-eda-predicting-optimal-fertilizers

In [3]:
import pickle
import os
# ^^^ pyforest auto-imports - don't write above this line
import pandas as pd
import numpy as np
from termcolor import colored

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler , LabelEncoder


#modeles :
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import RidgeClassifier, LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from xgboost import XGBClassifier


#import metrics
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from sklearn.model_selection import cross_val_score


import time
import pickle


from termcolor import colored
import os
import warnings
warnings.filterwarnings('ignore')

In [1]:
os.listdir()

<IPython.core.display.Javascript object>

['.ipynb_checkpoints',
 '1_EDA_Predicting_Optimal_Fertilizers.ipynb',
 '2_Model_Predicting_Optimal_Fertilizers.ipynb',
 'feature_encoder.pkl',
 'StandardScaler.pkl',
 'targetencoder.pkl']

In [4]:
#chargement fichier : 
path  = "D:/Etude_Data_science/Kaggle_competition/16_Predicting_Optimal_Fertilizers/notebook"
with open(f"{path}/StandardScaler.pkl", 'rb') as file:
    std = pickle.load(file)
with open(f"{path}/feature_encoder.pkl", 'rb') as file:
    feature_encoder = pickle.load(file)
with open(f"{path}/targetencoder.pkl", 'rb') as file:
    targetencoder = pickle.load(file)
    
# print(f"Données chargées depuis {colored('scaler.pkl','blue')} : {colored(scaler,'green',attrs=['bold'])}")

In [258]:
train_preprocessed = pd.read_csv(path+"//train_preprocessed.csv",index_col=0)
df = train_preprocessed.copy()
data_path = 'D:\\etude_data_science\\Kaggle_competition\\16_Predicting_Optimal_Fertilizers\\data\\'
test = pd.read_csv(data_path+"test.csv")
sample = pd.read_csv(data_path+"sample_submission.csv")

In [21]:
df.head()

,id,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,0,1.364188,1.347532,-0.778377,1,8,1.150999,-0.950608,-1.301500,4
1,1,-1.118810,1.197100,1.680146,4,4,0.616080,-0.603706,-0.248708,4
2,2,-0.622210,0.294510,-1.117484,4,4,0.081160,0.436998,-0.410676,2
3,3,0.867589,0.144078,0.747603,4,0,1.418459,0.436998,-1.382484,0
4,4,0.867589,-0.457648,-0.184941,3,6,1.240152,-1.297509,-0.410676,5


In [295]:
test.head()

,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous
0,31,70,52,Sandy,Wheat,34,11,24
1,27,62,45,Red,Sugarcane,30,14,15
2,28,72,28,Clayey,Ground Nuts,14,15,4
3,37,53,57,Black,Ground Nuts,18,17,36
4,31,55,32,Red,Pulses,13,19,14


In [269]:
df = df.drop("id", axis = 1)
id_test = test["id"]
test = test.drop("id",axis = 1)

##### Encodage des features de test :

In [ ]:
var_qual_feat = [i for i in feature_encoder.keys()]
var_cont = [i for i in df.select_dtypes(include=float)]
target = [i for i in df if i not in (var_cont + var_qual_feat )]

In [296]:
feature_encoder

{'Soil Type': LabelEncoder(), 'Crop Type': LabelEncoder()}

In [298]:
#Parcours du dictionnaire contenant les LabelEncoder de chaque feature :
for i in var_qual_feat:
    test[i] = feature_encoder[i].transform(test[i]) #Encodage par feature

In [303]:
test.head()
#Données correctement encodés

,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous
0,31,70,52,4,10,34,11,24
1,27,62,45,3,8,30,14,15
2,28,72,28,1,2,14,15,4
3,37,53,57,0,2,18,17,36
4,31,55,32,3,7,13,19,14


Les soumissions sont évaluées selon la précision moyenne à 3 (MAP@3) :

In [315]:
y = df['Fertilizer Name'] 
X = df.drop(['Fertilizer Name'],axis=1)
X_train, X_test, y_train,y_test = train_test_split(X, y,test_size = 0.2, random_state =42,stratify=y)


In [316]:
#instanciation modèle :
model = XGBClassifier(
    objective='multi:softprob', #softprob pour classification multiclass
    num_class=len(np.unique(y_train)),#indique explicitement nombre de classe
#     n_estimators=3200,
#     learning_rate=0.045,         
#     max_depth=7,                
#     colsample_bytree=0.6,       
#     colsample_bylevel=0.8,      
#     subsample=0.8,
)

#Entrainement : 
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None, num_class=7,
              num_parallel_tree=None, ...)

In [317]:
#Récupération des probabilités de prédictions : 
y_pred_probs = model.predict_proba(X_test)
#récupération prédiction :
y_pred = model.predict(X_test)

#### Explication pour les prédictions : 

In [318]:
print("shape : ", y_pred_probs.shape) #(observation, classes distinctes de la target)
print()
first_obs = y_pred_probs[0]
print("Probabilité de prediction : \n", first_obs)
print()
print("Classes :\n", targetencoder.classes_)

shape :  (150000, 7)

Probabilité de prediction : 
 [0.15073733 0.20225906 0.10215581 0.16125892 0.12258421 0.10869613
 0.15230854]

Classes :
 ['10-26-26' '14-35-14' '17-17-17' '20-20' '28-28' 'DAP' 'Urea']


- Premièrement avec shape nous voyons le nombre d'observation couplé au nombre de classe distinct : notre target prends  7 valeur différentes
- Nous avons affiché ensuite la 1ere observation, on se trouve avec une liste qui contient les probabilités d'appartenance à chaque classe
- par exemple x% d'appartenir à la classe 0, y% la classe 1 etc..


In [319]:
print(f"Pour la première prédiction, nous avons obtenu les % suivants :")
pourcent_firstobs = [f"{np.around(i*100,2)} %"  for i in first_obs]
print(pourcent_firstobs)
print()
print(f"La prédiction la plus élevée est la classe : {np.argmax(first_obs)}")
classe_firstobs = np.argmax(first_obs)
print(f"Cette classe correspond à la valeur original : {targetencoder.classes_[classe_firstobs]}")
print()
print(f"Si nous regardons non pas le predict_proba mais le predict :")
print(f"Nous obtenons bien la prédiction : {targetencoder.inverse_transform(y_pred[[0]])}")

Pour la première prédiction, nous avons obtenu les % suivants :
['15.07 %', '20.23 %', '10.22 %', '16.13 %', '12.26 %', '10.87 %', '15.23 %']

La prédiction la plus élevée est la classe : 1
Cette classe correspond à la valeur original : 14-35-14

Si nous regardons non pas le predict_proba mais le predict :
Nous obtenons bien la prédiction : ['14-35-14']


- L'exemple ci-dessus permet d'illustrer le fonctionnement des prédictions du modèle couplé aux probabilités de prédictions 

- Cependant ici, nous voulons récupérer les **3 prédictions les plus élevées pour chaque observations**
- Le np.argmax ne permet d'obtenir que la position de la prédiciton la plus élevée. Nous allons donc utiliser une autre méthode de numpy : **argsort**
    - argsort donne un nouveau tableau qui contient les positions (les indices) des éléments de notre tableau original, mais dans l'ordre où ils devraient apparaître si le tableau était trié.
    
##### Reprenons notre première observation pour illustrer : 

In [320]:
print("Prédiction pour la 1ere observation :")
print(pourcent_firstobs)
print()
print("Ordre dans lequel les indices devraient être pour les avoir")
print("dans l'ordre croissant :")
print()
print(np.argsort(first_obs))

Prédiction pour la 1ere observation :
['15.07 %', '20.23 %', '10.22 %', '16.13 %', '12.26 %', '10.87 %', '15.23 %']

Ordre dans lequel les indices devraient être pour les avoir
dans l'ordre croissant :

[2 5 4 0 6 3 1]


- Dans le argsort, la 1ere valeur obtenue correspond à l'élément le plus petit de la liste contenant les % de probabilité
- Le argsort classe les indices du tableau original dans l'ordre croissant 

#### Appliquons cela pour récupérer les 3 prédictions les plus élevées pour chaque observation :
- Pour cela, nous devrons classe le tableau dans l'ordre **décroissant**

In [321]:
np.argsort(y_pred_probs, axis=1) # permet d'avoir le classement des prédictions
#axis = 1 pour que le classement dans les colonnes de chaque liste
#et non pas entre les liste

array([[2, 5, 4, ..., 6, 3, 1],
       [6, 5, 3, ..., 4, 0, 1],
       [6, 1, 5, ..., 0, 2, 4],
       ...,
       [6, 2, 5, ..., 3, 1, 0],
       [5, 6, 3, ..., 1, 4, 0],
       [6, 2, 5, ..., 4, 0, 3]], dtype=int64)

- Pour rappel nous avions vu dans les cellules précédentes que le shape était un tuple (observation, nombre de classes)
- Ici nous voulons récupérer **toutes les observations** mais uniquement les **3 classes** avec les plus haute probabilités de prédictions
- Donc nous allons effectuer un premier slice **[:,-3:]**
- Ensuite on va utiliser un autre slice **[:,::-1]** qui permettra de passer de l'ordre **croissant  à décroissant**
    - le "[:" permet de selectionner toutes les lignes
    - le "::-1" d'agir sur toutes les colonnes de chaque ligne (donc chaque classe pour toutes les observations) et d'inverser l'ordre grâce au "-1"
    
* Le code suivant permet donc, pour chaque observations, d'obtenir les 3 classes avec la plus haute probabilité de prédiction ( [:,-3:] ). 
- Sauf que ces prédictions sont classées dans l'ordre **croissant**
- En utilisant [:,;;-1] on va, pour chaque observation obtenue, inverser l'ordre pour les avoir dans l'ordre **décroissant**

In [322]:
#Récupération du top 3 prédiction dans l'ordre décroissant  : 
top_3_preds = np.argsort(y_pred_probs, axis=1)[:, -3:][:, ::-1]  
#Affichons 5 premières prédictions :
top_3_preds[:5] 

array([[1, 3, 6],
       [1, 0, 4],
       [4, 2, 0],
       [5, 3, 0],
       [0, 4, 2]], dtype=int64)

- Il faut aussi transformer y_test en liste de liste : pour chaque observation, le résultat doit être contenu dans une liste 

In [323]:
#transformation de y_test en liste : 
actual = [[label] for label in y_test]

#### Explication du MAP@3 :

- La fonction MAP@3 attend des les vraies valeurs (y_test) au même format que les prédictions, c'est à dire des listes

- Par exemple :
    - actual = [['A'], ['B'], ['C']]             # c'est notre y_test, il y a 1 bonne réponse par ligne qui correspond à la vraie prédiction
    - predicted = [['A', 'B', 'C'], ['C', 'B'], ['C', 'A']]  # C'est nos prédictions, nous y avons récupéré les 3 prédictions les plus suceptibles d'être la bonne prédiction


##### ➡️ Le mapk va ensuite comparer :

Est-ce que la vraie valeur (ex: 'A') est bien dans le top-3 prédits ?

Et si oui, à quelle position (plus c’est haut, mieux c’est).


#### Regardons directement sur nos données et les 5 premières lignes :

In [324]:
actual[:5]

[[3], [2], [3], [0], [4]]

In [325]:
top_3_preds[:5]

array([[1, 3, 6],
       [1, 0, 4],
       [4, 2, 0],
       [5, 3, 0],
       [0, 4, 2]], dtype=int64)

In [326]:
print(f"Pour y_test (actual), la vraie target de la première prédiction est : {actual[0]}")
print()
print("On regarde si cette valeur vraie se trouve dans l'une des 3 prédictions de la 1ere observation")
print("Si elle l'est, on regarde sa position, plus elle est élevée et mieux c'est ")
print()
print(f"La valeur {actual[0]} est dans l'une des 3 prédictions de la première observation ?")
print(f"{top_3_preds[0]}")
print("Si non => mauvaise prédiction")
print("Si oui, en position 1, meilleure prédiction (=score 1) si position 3, plus faible score (1/3 = .33)")
#et si en position 2 alors 1/2 = .5 de score

Pour y_test (actual), la vraie target de la première prédiction est : [3]

On regarde si cette valeur vraie se trouve dans l'une des 3 prédictions de la 1ere observation
Si elle l'est, on regarde sa position, plus elle est élevée et mieux c'est 

La valeur [3] est dans l'une des 3 prédictions de la première observation ?
[1 3 6]
Si non => mauvaise prédiction
Si oui, en position 1, meilleure prédiction (=score 1) si position 3, plus faible score (1/3 = .33)


- **Création de la fonction pour le calcul map@3** :
    - (code récupéré sur le notebook https://www.kaggle.com/code/satyaprakashshukl/predicting-optimal-fertilizers)


In [327]:
def mapk(actual, predicted, k=3):
    def apk(a, p, k):
        p = p[:k]
        score = 0.0
        hits = 0
        seen = set()
        for i, pred in enumerate(p):
            if pred in a and pred not in seen:
                hits += 1
                score += hits / (i + 1.0)
                seen.add(pred)
        return score / min(len(a), k)
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

Récupération du score map3 : 

In [328]:
map3_score = mapk(actual, top_3_preds)
print(f"✅ MAP@3 Score: {map3_score:.5f}")

✅ MAP@3 Score: 0.32923


#### Soumission des prédictions : 

In [329]:
test_probs = model.predict_proba(test)
top_3_preds = np.argsort(test_probs, axis=1)[:, -3:][:, ::-1]
top_3_labels = targetencoder.inverse_transform(top_3_preds.ravel()).reshape(top_3_preds.shape)
submission = pd.DataFrame({
    'id': id_test,
    'Fertilizer Name': [' '.join(row) for row in top_3_labels]
})
submission.to_csv('submission.csv', index=False)
print("✅ Submission file saved as 'submission.csv'")


✅ Submission file saved as 'submission.csv'


In [330]:
submission.head()

,id,Fertilizer Name
0,750000,14-35-14 20-20 DAP
1,750001,14-35-14 20-20 28-28
2,750002,10-26-26 14-35-14 28-28
3,750003,10-26-26 28-28 20-20
4,750004,14-35-14 20-20 28-28


# On peut essayer avec d'autres modèles